通过 Log-Sum-Exp 逼近最大值实现最大池化

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SoftMaxPoolAsConv(nn.Module):
    def __init__(self, channels, kernel_size=2, stride=2, beta=10):
        super().__init__()
        self.beta = beta

        # 定义卷积层用于“求和”
        # 这里的设置和平均池化的一样：Groups=Channels，Bias=False
        self.conv = nn.Conv2d(
            in_channels=channels,
            out_channels=channels,
            kernel_size=kernel_size,
            stride=stride,
            groups=channels,
            bias=False
        )

        # 将卷积核权重初始化为 1.0
        # 因为我们要计算的是 sum(e^x)，而不是 average(e^x)
        self.conv.weight.data.fill_(1.0)
        self.conv.weight.requires_grad = False

    def forward(self, x):
        # 步骤 1: 放大并指数化 (非线性部分)
        # e^(beta * x)
        x_exp = torch.exp(self.beta * x)

        # 步骤 2: 使用卷积层进行求和 (线性部分)
        # sum(...)
        x_sum = self.conv(x_exp)

        # 步骤 3: 取对数并缩小 (还原部分)
        # (1/beta) * ln(...)
        # 加一个极小值 1e-8 防止 log(0)
        output = (1 / self.beta) * torch.log(x_sum + 1e-8)

        return output

验证

In [3]:
# --- 验证测试 ---

# 创建简单数据
X = torch.tensor([
    [
        [1.0, 5.0],
        [2.0, 8.0]
    ]
]).unsqueeze(0) # (1, 1, 2, 2)

print("原始数据:")
print(X)

# 1. 真实的最大池化
true_max = F.max_pool2d(X, kernel_size=2)
print(f"\n真实 Max Pooling 结果: {true_max.item()}")

# 2. 使用卷积逼近 (Beta 越大越精确，但容易溢出)
# 尝试不同的 Beta 值
for b in [1, 10, 50]:
    approx_layer = SoftMaxPoolAsConv(channels=1, kernel_size=2, stride=2, beta=b)
    approx_res = approx_layer(X)
    print(f"卷积逼近结果 (beta={b}): {approx_res.item():.4f}")


原始数据:
tensor([[[[1., 5.],
          [2., 8.]]]])

真实 Max Pooling 结果: 8.0
卷积逼近结果 (beta=1): 8.0518
卷积逼近结果 (beta=10): 8.0000
卷积逼近结果 (beta=50): inf
